Connection to API

In [2]:
import http.client
import json

conn = http.client.HTTPSConnection("google-flights2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "ed1c9558dfmshbf838e6e8278748p101e3bjsn795dbe58be79",
    'x-rapidapi-host': "google-flights2.p.rapidapi.com"
}

Fetching the airports data:

In [3]:
countries = ['Italy', 'USA']
countries_airports = []

for country in countries:
    conn.request("GET", f"/api/v1/searchAirport?query={country}&language_code=en-US", headers=headers)

    res = conn.getresponse()
    data = res.read()
    data = data.decode("utf-8")
    response = json.loads(data)

    if response.get("status") and response.get("message") == "Success":
        country_data = {
            "country": country,
            "cities": {}
        }

        for city_data in response.get("data", []):
            city_name = city_data["city"]

            airports = [
                {"id": item["id"], "name": item["title"]}
                for item in city_data.get("list", [])
                if item["type"] == "airport"
            ]

            if airports:
                country_data["cities"][city_name] = airports

        countries_airports.append(country_data)
    else:
        print(f"API response indicates failure for country: {country}")

print(json.dumps(countries_airports, indent=2))

[
  {
    "country": "Italy",
    "cities": {
      "Rome": [
        {
          "id": "FCO",
          "name": "Leonardo da Vinci International Airport"
        },
        {
          "id": "CIA",
          "name": "Giovan Battista Pastine International Airport"
        }
      ],
      "Florence": [
        {
          "id": "FLR",
          "name": "Amerigo Vespucci Airport"
        },
        {
          "id": "PSA",
          "name": "Pisa International Airport"
        }
      ],
      "Venice": [
        {
          "id": "VCE",
          "name": "Venice Marco Polo Airport"
        },
        {
          "id": "TSF",
          "name": "Treviso Airport"
        }
      ],
      "Milan": [
        {
          "id": "MXP",
          "name": "Milano Malpensa Airport"
        },
        {
          "id": "BGY",
          "name": "Il Caravaggio International Airport"
        },
        {
          "id": "LIN",
          "name": "Milan Linate Airport"
        }
      ],
      "Naples"

Connection to prices API 

In [6]:
# EUR

conn.request("GET", "/api/v1/searchFlights?departure_id=MXP&arrival_id=FCO&outbound_date=2025-04-01&travel_class=ECONOMY&adults=1&show_hidden=1&currency=EUR&language_code=en-US&country_code=US", headers=headers)

res = conn.getresponse()
data = res.read()
response = json.loads(data)

print(json.dumps(response, indent=2))

{
  "status": true,
  "message": "Success",
  "timestamp": 1737657949383,
  "data": {
    "itineraries": {
      "topFlights": [
        {
          "departure_time": "01-04-2025 07:10 AM",
          "arrival_time": "01-04-2025 08:35 AM",
          "duration": {
            "raw": 85,
            "text": "1 hr 25 min"
          },
          "flights": [
            {
              "departure_airport": {
                "airport_name": "Milano Malpensa Airport",
                "airport_code": "MXP",
                "time": "2025-4-1 07:10"
              },
              "arrival_airport": {
                "airport_name": "Leonardo da Vinci International Airport",
                "airport_code": "FCO",
                "time": "2025-4-1 08:35"
              },
              "duration": {
                "raw": 85,
                "text": "1 hr 25 min"
              },
              "airline": "AEROITALIA SRL",
              "airline_logo": "https://www.gstatic.com/flights/airline_logos/